In [ ]:
import os
import random
import numpy as np
import librosa
import soundfile as sf
from pydub import AudioSegment


INPUT_DIR = "datas/positiva normalizada"
OUTPUT_DIR = "datas/wake_word_augmented"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Funções de aumento
def add_noise(y, noise_level=0.005):
    noise = np.random.randn(len(y))
    return y + noise_level * noise

def change_pitch(y, sr, n_steps):
    try:
        from librosa import AudioData
        
        x = AudioData(y, sr=sr)
        x_shifted = librosa.effects.pitch_shift(x, n_steps=n_steps)
        return x_shifted.get_array()
    except ImportError:
        # fallback para librosa < 0.10
        return librosa.effects.pitch_shift(y, sr=sr, n_steps=n_steps)

def change_speed(y, rate):
    return librosa.effects.time_stretch(y,rate=rate)

def change_volume(y, gain_dB):
    return y * (10 ** (gain_dB / 20))

# Processar todos os arquivos
for fname in os.listdir(INPUT_DIR):
    if not fname.lower().endswith(".wav"):
        continue

    mp3_path = os.path.join(INPUT_DIR, fname)
    base_name = os.path.splitext(fname)[0]

    # Ler MP3 com pydub e converter para array
    audio = AudioSegment.from_mp3(mp3_path).set_channels(1).set_frame_rate(16000)
    samples = np.array(audio.get_array_of_samples()).astype(np.float32) / 32768.0
    sr = 16000

    # Salvar original em WAV
    sf.write(os.path.join(OUTPUT_DIR, f"{base_name}_orig.wav"), samples, sr)

    for i in range(5):  # gera 5 variações por arquivo
        y_aug = samples.copy()

        if random.random() < 0.5:
            y_aug = add_noise(y_aug, noise_level=random.uniform(0.002, 0.01))

        if random.random() < 0.5:
            y_aug = change_pitch(y_aug, sr, n_steps=random.uniform(-1, 1))

        if random.random() < 0.5:
            y_aug = change_speed(y_aug, rate=random.uniform(0.9, 1.1))

        if random.random() < 0.5:
            y_aug = change_volume(y_aug, gain_dB=random.uniform(-3, 3))

        out_fname = os.path.join(OUTPUT_DIR, f"{base_name}_aug{i+1}.wav")
        sf.write(out_fname, y_aug, sr)

print("Aumento de dados concluído!")


Aumento de dados concluído!
